# ⚡ NLP Sentiment API – Build and Test with FastAPI
This notebook creates a REST API for the trained sentiment model using FastAPI.

## 🔧 Step 1: Install and import required libraries

In [ ]:
!pip install fastapi[all] transformers uvicorn --quiet

In [ ]:

from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import uvicorn


## 🧠 Step 2: Load model and tokenizer

In [ ]:

model_path = "model/sentiment-bert"  # path where model was saved

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

labels = ["negative", "positive"]


## 🌐 Step 3: Define FastAPI app

In [ ]:

app = FastAPI()

class InputText(BaseModel):
    text: str

@app.post("/predict")
def predict_sentiment(input_text: InputText):
    inputs = tokenizer(input_text.text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1).squeeze()
        confidence, prediction = torch.max(probs, 0)
    return {
        "label": labels[prediction],
        "confidence": round(confidence.item(), 3)
    }


## 🚀 Step 4: Run the API (only works in terminal)

In [ ]:

# Uncomment and run this in terminal or script, not in notebook
# uvicorn.run(app, host="0.0.0.0", port=8000)


## 🧪 Example curl test (in terminal)
```bash
curl -X POST http://localhost:8000/predict -H "Content-Type: application/json" \
     -d '{"text": "This is absolutely amazing!"}'
```